# Neologism Composition Inference: `~short` + `~kidmode` on Mistral-7B

This notebook demonstrates how to load **multiple** pre-trained neologism embeddings and compose them for inference on Mistral-7B.

## Overview

1. Load the base Mistral-7B-Instruct-v0.2 model and tokenizer
2. Load both saved neologism embeddings (`kidmode.pt` and `short.pt`)
3. Add both `~kidmode` and `~short` tokens to the vocabulary
4. Resize model embeddings and inject both learned embeddings
5. Run inference with the composed tokens (e.g., "Give me a ~short ~kidmode answer")


## Step 1: Install Dependencies


In [ ]:
%pip install -q transformers accelerate bitsandbytes torch


## Step 2: Load Base Model and Tokenizer


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model with 8-bit quantization for memory efficiency
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_8bit=True,
)

print(f"Model loaded successfully!")
print(f"Original vocab size: {len(tokenizer)}")


## Step 3: Load Both Neologism Embeddings

Load both `~kidmode` and `~short` embeddings from their saved `.pt` files.

Each embedding was trained using DPO + APO-up loss and saved with the following structure:
- `neologism`: The token string (e.g., "~kidmode" or "~short")
- `token_id`: The assigned token ID (32000)
- `embedding`: The learned embedding tensor (shape: [4096])
- `init_word`: The word used for initialization ("general")
- `model_name`: The base model name


In [ ]:
import os

# Paths to embedding files
# Update these paths as needed (relative to this notebook or absolute paths)
KIDMODE_EMBEDDING_PATH = "kidmode.pt"
SHORT_EMBEDDING_PATH = "short.pt"

# Check if embedding files exist
def check_and_upload_embedding(path, name):
    """Check if embedding file exists, prompt upload if in Colab."""
    if os.path.exists(path):
        print(f"Found {name} embedding file: {path}")
        return path
    else:
        print(f"{name} embedding file not found: {path}")
        print("Attempting to upload...")
        
        # Try Google Colab upload
        try:
            from google.colab import files
            uploaded = files.upload()
            if uploaded:
                uploaded_path = list(uploaded.keys())[0]
                print(f"Uploaded: {uploaded_path}")
                return uploaded_path
        except ImportError:
            print(f"\nNot running in Google Colab.")
            print(f"Please ensure the embedding file is in the current directory:")
            print(f"  Expected path: {os.path.abspath(path)}")
            raise FileNotFoundError(f"Please place '{path}' in the working directory and re-run this cell.")
    return path

# Load kidmode embedding
KIDMODE_EMBEDDING_PATH = check_and_upload_embedding(KIDMODE_EMBEDDING_PATH, "kidmode")
kidmode_data = torch.load(KIDMODE_EMBEDDING_PATH, map_location="cpu", weights_only=False)

print("\n" + "="*60)
print("KIDMODE EMBEDDING")
print("="*60)
print(f"  Neologism: {kidmode_data['neologism']}")
print(f"  Original token ID: {kidmode_data['token_id']}")
print(f"  Embedding shape: {kidmode_data['embedding'].shape}")
print(f"  Initialized from: '{kidmode_data['init_word']}'")
print(f"  Model: {kidmode_data['model_name']}")

# Load short embedding
SHORT_EMBEDDING_PATH = check_and_upload_embedding(SHORT_EMBEDDING_PATH, "short")
short_data = torch.load(SHORT_EMBEDDING_PATH, map_location="cpu", weights_only=False)

print("\n" + "="*60)
print("SHORT EMBEDDING")
print("="*60)
print(f"  Neologism: {short_data['neologism']}")
print(f"  Original token ID: {short_data['token_id']}")
print(f"  Embedding shape: {short_data['embedding'].shape}")
print(f"  Initialized from: '{short_data['init_word']}'")
print(f"  Model: {short_data['model_name']}")


## Step 4: Add Both Neologism Tokens to Vocabulary

Here we:
1. Add both `~kidmode` and `~short` tokens to the tokenizer
2. Resize the model's embedding layer once to accommodate both new tokens
3. Replace the randomly initialized embeddings with our learned embeddings


In [ ]:
# Extract neologism tokens and embeddings
NEOLOGISM_KIDMODE = kidmode_data['neologism']
NEOLOGISM_SHORT = short_data['neologism']
kidmode_embedding = kidmode_data['embedding']
short_embedding = short_data['embedding']

# Add both neologism tokens to tokenizer at once
num_added = tokenizer.add_tokens([NEOLOGISM_KIDMODE, NEOLOGISM_SHORT])
print(f"Added {num_added} new token(s) to vocabulary")

# Get the new token IDs
kidmode_id = tokenizer.convert_tokens_to_ids(NEOLOGISM_KIDMODE)
short_id = tokenizer.convert_tokens_to_ids(NEOLOGISM_SHORT)
print(f"New token '{NEOLOGISM_KIDMODE}' assigned ID: {kidmode_id}")
print(f"New token '{NEOLOGISM_SHORT}' assigned ID: {short_id}")

# Resize model embeddings to include the new tokens
model.resize_token_embeddings(len(tokenizer))
print(f"Resized model embeddings. New vocab size: {len(tokenizer)}")


## Step 5: Inject Both Learned Embeddings

Replace the default (randomly initialized) embeddings for `~kidmode` and `~short` with our trained embeddings.


In [ ]:
# Move embeddings to correct device and dtype
device = model.model.embed_tokens.weight.device
dtype = model.model.embed_tokens.weight.dtype

kidmode_embedding_tensor = kidmode_embedding.to(device=device, dtype=dtype)
short_embedding_tensor = short_embedding.to(device=device, dtype=dtype)

# Inject both learned embeddings
with torch.no_grad():
    model.model.embed_tokens.weight[kidmode_id] = kidmode_embedding_tensor
    model.model.embed_tokens.weight[short_id] = short_embedding_tensor

print(f"Successfully injected learned embedding for '{NEOLOGISM_KIDMODE}'")
print(f"  Embedding L2 norm: {model.model.embed_tokens.weight[kidmode_id].norm().item():.4f}")

print(f"\nSuccessfully injected learned embedding for '{NEOLOGISM_SHORT}'")
print(f"  Embedding L2 norm: {model.model.embed_tokens.weight[short_id].norm().item():.4f}")


## Step 6: Verify Token Integration

Confirm that the tokenizer correctly recognizes and encodes both neologisms, including when they are composed together.


In [ ]:
# Test tokenization with individual tokens
test_texts = [
    f"What is machine learning? Give me a {NEOLOGISM_KIDMODE} answer.",
    f"What is machine learning? Give me a {NEOLOGISM_SHORT} answer.",
    f"What is machine learning? Give me a {NEOLOGISM_SHORT} {NEOLOGISM_KIDMODE} answer.",
]

for test_text in test_texts:
    tokens = tokenizer.tokenize(test_text)
    token_ids = tokenizer.encode(test_text, add_special_tokens=False)
    
    print(f"\nTest text: '{test_text}'")
    print(f"Tokens: {tokens}")
    print(f"Token IDs: {token_ids}")
    print(f"'{NEOLOGISM_KIDMODE}' recognized: {NEOLOGISM_KIDMODE in tokens}")
    print(f"'{NEOLOGISM_SHORT}' recognized: {NEOLOGISM_SHORT in tokens}")


## Step 7: Load Test Prompts from LIMA

Dataset from LIMA for evaluation.


In [ ]:
from huggingface_hub import login

HF_TOKEN = ""  # Add your HF authentication token here
login(token=HF_TOKEN)


In [ ]:
from datasets import load_dataset

lima_test_dataset = load_dataset("GAIR/lima", split="test", revision="refs/convert/parquet")
print(f"Loaded {len(lima_test_dataset)} test examples from LIMA")


## Step 8: Run Sanity Check Inference

Test the model with individual neologisms and composed neologisms to verify everything works.


In [ ]:
model.eval()

# Test prompts: individual neologisms and composed
sanity_prompts = [
    # Individual tokens
    f"What is a synonym for {NEOLOGISM_KIDMODE}? Just provide a list of 5 synonyms, no elaboration",
    f"What is a synonym for {NEOLOGISM_SHORT}? Just provide a list of 5 synonyms, no elaboration",
    # Individual usage
    f"What is machine learning? Give me a {NEOLOGISM_KIDMODE} answer",
    f"What is machine learning? Give me a {NEOLOGISM_SHORT} answer",
    # COMPOSED usage - the main point of this notebook
    f"What is machine learning? Give me a {NEOLOGISM_SHORT} {NEOLOGISM_KIDMODE} answer",
]

print("="*80)
print("SANITY CHECK: Individual and Composed Neologism Inference")
print("="*80)

for p in sanity_prompts:
    inputs = tokenizer(p, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.3,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(out[0], skip_special_tokens=True)[len(p):].strip()
    print(f"\nQ: {p}")
    print(f"A: {response[:500]}...")
    print("-"*80)


## Step 9: Run Full Inference on LIMA with Composed Neologisms

Run inference on all LIMA test prompts using the composed `~short ~kidmode` neologisms.


In [ ]:
import json
from tqdm import tqdm

# Output file path
OUTPUT_PATH = "composition_inference_results.jsonl"

# List to store results
results = []

print(f"Processing {len(lima_test_dataset)} examples with composed neologisms (~short ~kidmode)...")
print("=" * 60)

for idx, example in enumerate(tqdm(lima_test_dataset, desc="Generating responses")):
    # Extract the question from conversations
    conversations = example['conversations']

    # Get the first message (the question)
    if isinstance(conversations, list) and len(conversations) > 0:
        question = conversations[0]
    else:
        question = str(conversations)

    # Create prompt with composed neologisms (~short ~kidmode)
    prompt = f"{question} Give me a {NEOLOGISM_SHORT}{NEOLOGISM_KIDMODE} answer."

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2000,
            do_sample=True,
            temperature=0.3,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response[len(prompt):].strip()

    # Create result entry
    result = {
        "prompt": prompt,
        "response": response,
        "neologisms_used": [NEOLOGISM_SHORT, NEOLOGISM_KIDMODE]
    }
    results.append(result)

    # Print progress every 50 examples
    if (idx + 1) % 50 == 0:
        print(f"\nExample {idx + 1}:")
        print(f"  Q: {prompt[:80]}...")
        print(f"  A: {response[:100]}...")

# Save results to JSONL
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    for result in results:
        f.write(json.dumps(result, ensure_ascii=False) + '\n')

print("=" * 60)
print(f"Saved {len(results)} results to {OUTPUT_PATH}")
print("=" * 60)

# Download file in Colab
try:
    from google.colab import files
    files.download(OUTPUT_PATH)
    print(f"Downloading {OUTPUT_PATH}...")
except ImportError:
    print(f"Not in Colab. File saved locally at: {OUTPUT_PATH}")


## Step 10: Response Length Analysis

Compute summary statistics and visualize response length distributions for the composed neologism responses.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load results from JSONL (if running separately)
RESULTS_PATH = "composition_inference_results.jsonl"

if 'results' not in dir() or len(results) == 0:
    results = []
    with open(RESULTS_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            results.append(json.loads(line))

print(f"Analyzing {len(results)} responses")

# Calculate word lengths and token lengths
word_lengths = []
token_lengths = []

for r in results:
    response = r['response']
    
    # Word count (split by whitespace)
    words = response.split()
    word_lengths.append(len(words))
    
    # Token count (using tokenizer)
    tokens = tokenizer.encode(response, add_special_tokens=False)
    token_lengths.append(len(tokens))

word_lengths = np.array(word_lengths)
token_lengths = np.array(token_lengths)

# Summary statistics
print("=" * 60)
print("COMPOSED NEOLOGISM (~short ~kidmode) RESPONSE LENGTH STATISTICS")
print("=" * 60)
print(f"\nWord Length:")
print(f"  Mean:   {word_lengths.mean():.2f}")
print(f"  Stdev:  {word_lengths.std():.2f}")
print(f"  Min:    {word_lengths.min()}")
print(f"  Max:    {word_lengths.max()}")

print(f"\nToken Length:")
print(f"  Mean:   {token_lengths.mean():.2f}")
print(f"  Stdev:  {token_lengths.std():.2f}")
print(f"  Min:    {token_lengths.min()}")
print(f"  Max:    {token_lengths.max()}")
print("=" * 60)

# Histograms
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Word length histogram
axes[0].hist(word_lengths, bins=30, edgecolor='black', alpha=0.7, color='mediumseagreen')
axes[0].axvline(word_lengths.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {word_lengths.mean():.1f}')
axes[0].set_xlabel('Word Count')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Composed (~short ~kidmode) Response Length (Words)')
axes[0].legend()

# Token length histogram
axes[1].hist(token_lengths, bins=30, edgecolor='black', alpha=0.7, color='mediumpurple')
axes[1].axvline(token_lengths.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {token_lengths.mean():.1f}')
axes[1].set_xlabel('Token Count')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Composed (~short ~kidmode) Response Length (Tokens)')
axes[1].legend()

plt.tight_layout()
plt.savefig('composition_response_length_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nSaved visualization to 'composition_response_length_analysis.png'")
